### Loading all the Documents

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

loader = DirectoryLoader(
    path="../datas/",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
)

documents = loader.load()

print(f"Loaded {len(documents)} documents")
print(f"\nFirst document content:\n{documents[0].page_content}")
print(documents[0].page_content[:500])  # Print first 500 characters

In [2]:
# Initializing test Spillter
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", " ", ""],
)

chunks = text_splitter.split_documents(documents=documents)

print(f"Created {len(chunks)} chunks")
print(f"\nFirst chunk content:\n{chunks[0].page_content}")
print(f"Content {chunks[0].page_content[:500]}")  # Print first 500 characters
print(f"Metadata {chunks[0].metadata}")  # Print metadata of the first chunk

Created 6 chunks

First chunk content:
Deep learning is a subset of machine learning that uses multi-layered neural networks to automatically learn hierarchical representations from data. A neural network is composed of interconnected nodes (“neurons”) organized in layers—input, hidden, and output—that transform inputs through learned weights and nonlinear activations. By training on large datasets with optimization methods like stochastic gradient descent, these models capture complex patterns for tasks such as image recognition,
Content Deep learning is a subset of machine learning that uses multi-layered neural networks to automatically learn hierarchical representations from data. A neural network is composed of interconnected nodes (“neurons”) organized in layers—input, hidden, and output—that transform inputs through learned weights and nonlinear activations. By training on large datasets with optimization methods like stochastic gradient descent, these models capture complex pat

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

### Loading the ChromaDB

In [38]:
from dotenv import load_dotenv
load_dotenv()

import os
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction


persist_directory = "./chroma_db"
client = chromadb.PersistentClient(
    path=persist_directory,
)

collection = client.get_or_create_collection(
    name="my_collection",
    embedding_function=OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-3-small"),
)
collection.add(
    ids=[str(i) for i in range(len(documents))],
    documents=documents,
)

print(f"Vector store created with {collection.count()} vectors")
print(f"Persisted at: {persist_directory}")

ValueError: Expected document to be a str, got page_content='Deep learning is a subset of machine learning that uses multi-layered neural networks to automatically learn hierarchical representations from data. A neural network is composed of interconnected nodes (“neurons”) organized in layers—input, hidden, and output—that transform inputs through learned weights and nonlinear activations. By training on large datasets with optimization methods like stochastic gradient descent, these models capture complex patterns for tasks such as image recognition, natural language understanding, and speech synthesis. Depth (many layers) enables the network to learn increasingly abstract features, often outperforming traditional methods when ample data and compute are available.' metadata={'source': '../datas/doc_2.txt'} in add.

In [30]:
query = "What is Types of Machine Learning?"

similer_docst = collection.query(
    query_texts=[query],
    n_results=2,
)
similer_docst

{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[]],
 'distances': [[]]}